이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 10.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=e750b1ebc07617ee7116f668ac626c816dbbe386ad7f473824a1e05975e37db1
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
!cd /usr/local/lib/python3.10/dist-packages/pyspark/jars && wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.28/redshift-jdbc42-2.1.0.28.jar

--2024-06-20 05:22:49--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.28/redshift-jdbc42-2.1.0.28.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.236.128, 54.231.232.152, 52.217.116.32, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.236.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1077030 (1.0M) [application/java-archive]
Saving to: ‘redshift-jdbc42-2.1.0.28.jar’

redshift-jdbc42-2.1 100%[===================>]   1.03M  2.33MB/s    in 0.4s    

2024-06-20 05:22:50 (2.33 MB/s) - ‘redshift-jdbc42-2.1.0.28.jar’ saved [1077030/1077030]



**Spark Session**을 하나 만든다

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL 저장하기") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.sql.adaptive.enabled", False) \
    .enableHiveSupport() \
    .getOrCreate()

Redshift와 연결해서 DataFrame으로 로딩하기

In [4]:
url = "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=****"

df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

In [5]:
join_expr = df_user_session_channel.sessionid == df_session_timestamp.sessionid
df_join = df_user_session_channel.join(df_session_timestamp, join_expr, "inner")

In [6]:
df_join.show(10)

+------+--------------------+--------+--------------------+--------------------+
|userid|           sessionid| channel|           sessionid|                  ts|
+------+--------------------+--------+--------------------+--------------------+
|  1501|0135456d6a3c1051f...|  Google|0135456d6a3c1051f...|2019-09-24 14:49:...|
|   876|01a416a7e28d0d229...|Facebook|01a416a7e28d0d229...|2019-05-26 14:23:...|
|   243|0226aa5193c66d990...|  Google|0226aa5193c66d990...|2019-07-01 23:04:...|
|  2776|029bf49b584c641f0...|Facebook|029bf49b584c641f0...|2019-11-11 20:37:...|
|   939|02b8d6c2775b756de...|  Google|02b8d6c2775b756de...|2019-09-01 15:29:...|
|  2133|02ea66ee24d57e285...| Organic|02ea66ee24d57e285...| 2019-10-28 22:50:40|
|   771|02f6364bf207d6837...|   Naver|02f6364bf207d6837...|2019-09-07 18:24:...|
|  1961|0302915889fa38fe5...| Youtube|0302915889fa38fe5...| 2019-11-29 15:16:49|
|  2139|039cc47f960921e3c...|   Naver|039cc47f960921e3c...| 2019-08-01 18:19:34|
|   599|03ce0331cbd983f16...

In [9]:
spark.sql("DROP TABLE IF EXISTS bk_usc")

DataFrame[]

In [8]:
spark.sql("DROP TABLE IF EXISTS bk_st")

DataFrame[]

In [12]:
df_user_session_channel.write.mode("overwrite").bucketBy(3, "sessionid").saveAsTable("bk_usc")
df_session_timestamp.write.mode("overwrite").bucketBy(3, "sessionid").saveAsTable("bk_st")

In [13]:
df_bk_usc = spark.read.table("bk_usc")
df_bk_st = spark.read.table("bk_st")

In [14]:
join_expr2 = df_bk_usc.sessionid == df_bk_st.sessionid
df_join2 = df_bk_usc.join(df_bk_st, join_expr2, "inner")

In [15]:
df_join2.show(10)

+------+--------------------+---------+--------------------+--------------------+
|userid|           sessionid|  channel|           sessionid|                  ts|
+------+--------------------+---------+--------------------+--------------------+
|  1491|00029153d12ae1c9a...|  Organic|00029153d12ae1c9a...|2019-10-18 14:14:...|
|  1667|000958fdaefe0dd06...|Instagram|000958fdaefe0dd06...|2019-11-02 14:52:...|
|   468|000a3f777828d2cdb...| Facebook|000a3f777828d2cdb...| 2019-11-29 16:33:55|
|   780|000c076c390a4c357...|    Naver|000c076c390a4c357...|2019-05-02 15:24:...|
|   711|00106ac9184e7d54b...|  Organic|00106ac9184e7d54b...|2019-07-25 18:14:...|
|  1653|0012a83c1e6cda9d7...|   Google|0012a83c1e6cda9d7...| 2019-08-07 16:08:41|
|    64|001a3678cacf4ea11...|Instagram|001a3678cacf4ea11...|2019-10-04 19:57:...|
|  2027|001c4bab90e1fbf3a...| Facebook|001c4bab90e1fbf3a...| 2019-11-20 20:23:10|
|   197|001d3439223b7bb23...|    Naver|001d3439223b7bb23...|2019-08-14 15:14:...|
|  1513|001f35b8